In [ ]:
# Tuodaan PuLP kirjasto
from pulp import *
# Jos tulee virheilmoitus niin anna ensin komento "pip install pulp"

#Data
kuljetuskustannukset=[[53,95,136,160],
[60,120,132,140],
[210, 190, 89, 71]]

#Tuotantokapasiteetit tehtailla
kapasiteetit=[10200, 2000, 4100]

#Kysynnät varastoilla
kysynnat = [2500, 3000, 5500, 5000]

#Indeksit
tehtaat = range(3)
varastot= range(4)

#Luodaan (minimoiva) lineaarinen optimointi malli PuLP:ssa
malli = LpProblem("Kuljetusmalli", LpMinimize)

#Luodaan päätösmuuttujat x_ij: "kuljetetaan x_ij tonnia kartonkia tehtaalta i varastoon j"
x = LpVariable.dicts("Reitti",(tehtaat,varastot),0,None)

#Lisätään kohdefunktio eli summa kaikkien reittien kuljetuskustannuksista
malli += (lpSum([kuljetuskustannukset[i][j]*x[i][j] for i in tehtaat for j in varastot]), "Kuljetuskustannukset")

#Tuotantokapasiteettirajoitteet tehtailla
for i in tehtaat:
    malli += (lpSum([x[i][j] for j in varastot])<=kapasiteetit[i], "Tuotanto_tehtaalla_%s"%(i+1))

#Kysyntärajoitteet varastoilla
for j in varastot:
    malli += (lpSum([x[i][j] for i in tehtaat])==kysynnat[j], "Kysyntä_varastolla_%s"%(j+1))
    
#Ratkaistaan malli, eli etsitään optimaalinen ratkaisu (tai todetaan että mallille ei ole käypää ratkaisua)     
malli.solve()

# Tulostetaan päätösmuuttujien arvot optimiratkaisussa
kohdefunktionarvo = 0
for i in tehtaat:
    for j in varastot:
        print("Kuljetetaan %s tonnia kartonkia tehtaalta %d varastolle %a." % (x[i][j].varValue, (i+1), (j+1))) 
        kohdefunktionarvo = kohdefunktionarvo + kuljetuskustannukset[i][j]*x[i][j].varValue
    print("---")
    
# Tulostetaan optimaalinen kohdefunktion arvo 
print("Kokonaiskuljetuskustannus = %s " % kohdefunktionarvo)

